In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages',
                     '../src']
sys.prefix = '/home/joaom/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

In [1]:
from pathlib import Path
home = str(Path.home())
import sys
sys.path =  [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']

In [2]:
import awswrangler as wr
import pandas as pd
import boto3

In [4]:
from h3 import h3
from shapely import wkt
import geojson
from copy import deepcopy

In [5]:
metadata = pd.read_csv('http://tiny.cc/idb-traffic-metadata')

In [6]:
def wkt_to_geojson(s):
    return geojson.Feature(geometry=wkt.loads(s), properties={}).geometry

def to_wkt(x):
    x = x + [x[0]]
    return "polygon" + str(x).replace("], [", ",").replace(", ", " ").replace(
        "[", "("
    ).replace("]", ")")

In [7]:
metadata['geojson'] = metadata['region_shapefile_wkt'].apply(wkt_to_geojson)

In [11]:
def get_cells(m):

    df = pd.DataFrame()
    for resolution in [9, 10]:

        temp = pd.Series(list(h3.polyfill(m['geojson'], resolution)), name='id').to_frame()
        temp['resolution'] = resolution
        temp['wkt'] = temp['id'].apply(lambda x: to_wkt(h3.h3_to_geo_boundary(x)))

        df = pd.concat([df, deepcopy(temp)])
    
    df['region_slug'] = m['region_slug']
    
    return df

In [20]:
m = metadata.query('region_slug == "bogota"').iloc[0]
get_cells(m)

,id,resolution,wkt,region_slug
0,89e76d6a88bffff,9,polygon((-74.08429571498272 4.7927174574629205...,bogota
1,89e76d61d03ffff,9,"polygon((-74.13635652830591 4.558247415568863,...",bogota
2,89e76d60177ffff,9,"polygon((-74.17124504644951 4.540162300852441,...",bogota
3,89e76d61ba3ffff,9,"polygon((-74.10088594151274 4.539676375219948,...",bogota
4,89e76d6032fffff,9,"polygon((-74.15107139611781 4.561932780482447,...",bogota
...,...,...,...,...
14007,8ae76d61b177fff,10,"polygon((-74.09825581732875 4.558449088749691,...",bogota
14008,8ae76d683b77fff,10,"polygon((-74.02709614183726 4.88643735217781,-...",bogota
14009,8ae76d6e458ffff,10,"polygon((-74.10758979503686 4.462549582878939,...",bogota
14010,8ae76d6f50affff,10,"polygon((-74.07174978024169 4.524482695069525,...",bogota


In [21]:
config = dict(
    athena_database='spd_sdv_waze_corona',
    bucket='iadbprod-public-stata-as-a-service',
    prefix='spd-sdv-omitnik-waze/corona',
    raw_table='grid',
    name='test_grid',
    slug='dev',
    mode='overwrite_partitions'
)

In [22]:
res = wr.s3.to_parquet(
    df=get_cells(m),
    path="s3://{bucket}/{prefix}/{slug}/{raw_table}/{name}".format(**config),
    dataset=True,
    database=config["athena_database"],
    table="{slug}_{raw_table}_{name}".format(**config),
    mode=config["mode"],
    partition_cols=["region_slug"],
    boto3_session=boto3.Session(region_name="us-east-1"),
)

In [ ]:
query = """
with t as (
      select 
            length,
            year(retrievaltime) as year,
            month(retrievaltime) as month,
            day(retrievaltime) as day,
            day_of_week(retrievaltime) as dow,
                        date_parse(format_datetime(date_add('minute', 
                                                                                cast(date_diff('minute',
                                                                                        timestamp '2015-01-01 00:00:00', retrievaltime) / 5 as bigint) * 5,
                                                                                        timestamp '2015-01-01 00:00:00'), 'H:m'), '%H:%i') as time,
                        row_number() over (partition by uuid, year(retrievaltime), month(retrievaltime), day(retrievaltime),
                                                                        date_parse(format_datetime(date_add('minute', 
                                                                                cast(date_diff('minute',
                                                                                        timestamp '2015-01-01 00:00:00', retrievaltime) / 5 as bigint) * 5,
                                          timestamp '2015-01-01 00:00:00'), 'H:m'), '%H:%i') order by retrievaltime) n_row
      from spd_sdv_waze_corona.dev_daily_daily_raw
      where 
         
            country = 'CO' and
                st_intersects(
              st_polygon('Polygon ((-74.04208330493199242 4.93291779846285294, -74.01708330657054091 4.93291779846285294, -74.01708330657054091 4.92458446456020482, -74.00874997378338094 4.92458446456020482, -74.00874997378338094 4.90791779675490858, -74.01708330657054091 4.90791779675490858, -74.01708330657054091 4.89958446285226046, -74.02541663935770089 4.89958446285226046, -74.02541663935770089 4.88291779504697132, -74.01708330657054091 4.88291779504697132, -74.01708330657054091 4.84958445943637884, -74.02541663935770089 4.84958445943637884, -74.02541663935770089 4.84125112553373071, -74.03374997214484665 4.84125112553373071, -74.03374997214484665 4.83291779163108259, -74.01708330657054091 4.83291779163108259, -74.01708330657054091 4.82458445772843447, -74.00874997378338094 4.82458445772843447, -74.00874997378338094 4.81625112382578635, -74.00041664099623517 4.81625112382578635, -74.00041664099623517 4.80791778992314534, -74.00874997378338094 4.80791778992314534, -74.00874997378338094 4.79125112211784909, -74.01708330657054091 4.79125112211784909, -74.01708330657054091 4.78291778821520097, -74.00874997378338094 4.78291778821520097, -74.00874997378338094 4.77458445431255285, -74.01708330657054091 4.77458445431255285, -74.01708330657054091 4.76625112040990473, -74.00874997378338094 4.76625112040990473, -74.00874997378338094 4.73291778479931935, -74.00041664099623517 4.73291778479931935, -74.00041664099623517 4.72458445089667123, -74.00874997378338094 4.72458445089667123, -74.00874997378338094 4.69125111528607874, -74.00041664099623517 4.69125111528607874, -74.00041664099623517 4.66625111357813438, -74.00874997378338094 4.66625111357813438, -74.00874997378338094 4.65791777967549336, -74.01708330657054091 4.65791777967549336, -74.01708330657054091 4.64958444577284524, -74.02541663935770089 4.64958444577284524, -74.02541663935770089 4.64125111187019712, -74.00874997378338094 4.64125111187019712, -74.00874997378338094 4.61625111016225276, -74.01708330657054091 4.61625111016225276, -74.01708330657054091 4.60791777625960464, -74.02541663935770089 4.60791777625960464, -74.02541663935770089 4.57458444064901926, -74.01708330657054091 4.57458444064901926, -74.01708330657054091 4.56625110674637114, -74.02541663935770089 4.56625110674637114, -74.02541663935770089 4.55791777284372301, -74.03374997214484665 4.55791777284372301, -74.03374997214484665 4.54958443894107489, -74.04208330493199242 4.54958443894107489, -74.04208330493199242 4.54125110503842677, -74.05041663771915239 4.54125110503842677, -74.05041663771915239 4.53291777113577865, -74.05874997050631237 4.53291777113577865, -74.05874997050631237 4.49958443552519327, -74.06708330329345813 4.49958443552519327, -74.06708330329345813 4.48291776771989703, -74.07541663608060389 4.48291776771989703, -74.07541663608060389 4.46625109991460079, -74.08374996886777808 4.46625109991460079, -74.08374996886777808 4.45791776601195266, -74.12541663280353532 4.45791776601195266, -74.12541663280353532 4.44958443210930454, -74.13374996559068109 4.44958443210930454, -74.13374996559068109 4.45791776601195266, -74.15041663116500104 4.45791776601195266, -74.15041663116500104 4.46625109991460079, -74.1587499639521468 4.46625109991460079, -74.1587499639521468 4.48291776771989703, -74.16708329673930677 4.48291776771989703, -74.16708329673930677 4.49125110162254515, -74.17541662952646675 4.49125110162254515, -74.17541662952646675 4.50791776942784139, -74.18374996231361251 4.50791776942784139, -74.18374996231361251 4.51625110333048951, -74.19208329510075828 4.51625110333048951, -74.19208329510075828 4.54125110503842677, -74.21708329346222399 4.54125110503842677, -74.21708329346222399 4.53291777113577865, -74.22541662624938397 4.53291777113577865, -74.22541662624938397 4.52458443723313053, -74.25874995739800966 4.52458443723313053, -74.25874995739800966 4.53291777113577865, -74.28374995575946116 4.53291777113577865, -74.28374995575946116 4.54125110503842677, -74.29208328854662113 4.54125110503842677, -74.29208328854662113 4.55791777284372301, -74.28374995575946116 4.55791777284372301, -74.28374995575946116 4.56625110674637114, -74.27541662297230118 4.56625110674637114, -74.27541662297230118 4.57458444064901926, -74.28374995575946116 4.57458444064901926, -74.28374995575946116 4.58291777455166738, -74.29208328854662113 4.58291777455166738, -74.29208328854662113 4.5912511084543155, -74.28374995575946116 4.5912511084543155, -74.28374995575946116 4.59958444235695652, -74.26708329018515542 4.59958444235695652, -74.26708329018515542 4.60791777625960464, -74.25874995739800966 4.60791777625960464, -74.25874995739800966 4.61625111016225276, -74.25041662461083547 4.61625111016225276, -74.25041662461083547 4.62458444406490088, -74.23374995903654394 4.62458444406490088, -74.23374995903654394 4.632917777967549, -74.22541662624938397 4.632917777967549, -74.22541662624938397 4.64958444577284524, -74.21708329346222399 4.64958444577284524, -74.21708329346222399 4.65791777967549336, -74.20041662788793246 4.65791777967549336, -74.20041662788793246 4.66625111357813438, -74.18374996231361251 4.66625111357813438, -74.18374996231361251 4.68291778138343062, -74.19208329510075828 4.68291778138343062, -74.19208329510075828 4.69125111528607874, -74.20041662788793246 4.69125111528607874, -74.20041662788793246 4.68291778138343062, -74.20874996067507823 4.68291778138343062, -74.20874996067507823 4.69125111528607874, -74.24208329182368971 4.69125111528607874, -74.24208329182368971 4.69958444918872686, -74.25041662461083547 4.69958444918872686, -74.25041662461083547 4.70791778309137499, -74.25874995739800966 4.70791778309137499, -74.25874995739800966 4.71625111699402311, -74.28374995575946116 4.71625111699402311, -74.28374995575946116 4.72458445089667123, -74.29208328854662113 4.72458445089667123, -74.29208328854662113 4.74125111870196037, -74.28374995575946116 4.74125111870196037, -74.28374995575946116 4.74958445260460849, -74.25041662461083547 4.74958445260460849, -74.25041662461083547 4.74125111870196037, -74.24208329182368971 4.74125111870196037, -74.24208329182368971 4.73291778479931935, -74.22541662624938397 4.73291778479931935, -74.22541662624938397 4.74125111870196037, -74.20041662788793246 4.74125111870196037, -74.20041662788793246 4.73291778479931935, -74.19208329510075828 4.73291778479931935, -74.19208329510075828 4.72458445089667123, -74.18374996231361251 4.72458445089667123, -74.18374996231361251 4.71625111699402311, -74.16708329673930677 4.71625111699402311, -74.16708329673930677 4.72458445089667123, -74.1587499639521468 4.72458445089667123, -74.1587499639521468 4.73291778479931935, -74.15041663116500104 4.73291778479931935, -74.15041663116500104 4.74125111870196037, -74.14208329837785527 4.74125111870196037, -74.14208329837785527 4.74958445260460849, -74.13374996559068109 4.74958445260460849, -74.13374996559068109 4.76625112040990473, -74.12541663280353532 4.76625112040990473, -74.12541663280353532 4.77458445431255285, -74.11708330001638956 4.77458445431255285, -74.11708330001638956 4.79125112211784909, -74.10874996722922958 4.79125112211784909, -74.10874996722922958 4.79958445602049721, -74.09208330165492384 4.79958445602049721, -74.09208330165492384 4.80791778992314534, -74.08374996886777808 4.80791778992314534, -74.08374996886777808 4.81625112382578635, -74.09208330165492384 4.81625112382578635, -74.09208330165492384 4.83291779163108259, -74.08374996886777808 4.83291779163108259, -74.08374996886777808 4.84958445943637884, -74.09208330165492384 4.84958445943637884, -74.09208330165492384 4.86625112724167508, -74.08374996886777808 4.86625112724167508, -74.08374996886777808 4.88291779504697132, -74.07541663608060389 4.88291779504697132, -74.07541663608060389 4.89125112894961234, -74.06708330329345813 4.89125112894961234, -74.06708330329345813 4.89958446285226046, -74.05874997050631237 4.89958446285226046, -74.05874997050631237 4.90791779675490858, -74.05041663771915239 4.90791779675490858, -74.05041663771915239 4.92458446456020482, -74.04208330493199242 4.92458446456020482, -74.04208330493199242 4.93291779846285294))'),
              st_line(line))
      
                  )
select 
        year,
        month, 
        dow, 
        day, 
        sum(length) as sum_length
from t
where n_row = 1
group by  year, month, dow, day
"""

In [23]:
pd.read_csv('https://idbg.sharepoint.com/sites/CoronavirusImpactDashboard/Shared%20Documents/General/Dashboard/Data%20Raw/Public%20Transport/S%C3%A3o%20Paulo/sp_busridership_baselineJanuary.csv')

HTTPError: HTTP Error 403: FORBIDDEN

In [24]:
daily = pd.read_csv('http://tiny.cc/idb-traffic-daily')

In [31]:
a = daily.query('region_type == "state"')['region_name'].unique()
a.sort()
a

array(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara',
       'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao',
       'Mato Grosso', 'Minas Gerais', 'Para', 'Pernambuco', 'Piaui',
       'Rio De Janeiro', 'Rio Grande Do Norte', 'Rio Grande Do Sul',
       'Rondonia', 'Roraima', 'Santa Catarina', 'Sao Paulo', 'Sergipe',
       'Tocantins'], dtype=object)

In [ ]:
metadata.query('region_type == "state"')